# Derive insights on Olympics data using Python Pandas 
### <font color='blue'> Expose an integration point using websockets for orchestration with Node-RED.</font>

## 1. Setup
To prepare your environment, you need to install some packages.

### 1.1 Install the necessary packages

You need the latest versions of these packages:<br>
- websocket-client: is a python client for the Websockets.<br>
- python-swiftclient: is a python client for the Swift API.<br><br>

** Install the websocket client: **

In [13]:
!pip install websocket-client

In [14]:
!pip install ibm-cos-sdk

### 1.2 Import packages and libraries

Import the packages and libraries that you'll use:

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import json
import websocket
import _thread
import time
import ibm_boto3
from botocore.client import Config
import codecs
from io import StringIO

## 2. Configuration

Add configurable items of the notebook below

### 2.1 Add your service credentials for Object Storage

You must create Object Storage service on IBM Cloud.
To access data in a file in Object Storage, you need the Object Storage authentication credentials.
Insert the Object Storage authentication credentials as <i><b>credentials_1</b></i> in the following cell after 
removing the current contents in the cell. 

In [16]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': '',
    'IAM_SERVICE_ID': '',
    'ENDPOINT': '',
    'IBM_AUTH_ENDPOINT': '',
    'BUCKET': '',
    'FILE': ''
}

### 2.3 Global Variables

Add global variables.

In [17]:
olympics_data_filename = 'olympics.csv'
dictionary_data_filename = 'dictionary.csv'

# 3. Persistence and Storage

### 3.1 Configure Object Storage Client

cos = ibm_boto3.client('s3',
                    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                    ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                    config=Config(signature_version='oauth'),
                    endpoint_url=credentials_1['ENDPOINT'])

def get_file(filename):
    '''Retrieve file from Cloud Object Storage'''
    fileobject = cos.get_object(Bucket=credentials_1['BUCKET'], Key=filename)['Body']
    return fileobject

def load_string(fileobject):
    '''Load the file contents into a Python string'''
    text = fileobject.read()
    return text

def put_file(filename, filecontents):
    '''Write file to Cloud Object Storage'''
    resp = cos.put_object(Bucket=credentials_1['BUCKET'], Key=filename, Body=filecontents)
    return resp

# 4. Data 

### 4.1 Prepare data
Combine the olympics and dictionary data into a single dataframe:
- Read olympics data from Object Storage.<br>
- Rename columns<br>
- Populate the data in the dictionary to the Olympics data with a merge<br><br>

In [18]:
olympics = pd.read_csv(olympics_data_filename)
olympics = olympics.rename(columns = {'Country':'Code'})
olympics = olympics.rename(columns = {'Year':'Edition'})
dictionary = pd.read_csv(dictionary_data_filename)
olympics = pd.merge(olympics, dictionary, on='Code')
olympics.head()

,Edition,City,Sport,Discipline,Athlete,Code,Gender,Event,Medal,Country,Population,GDP per Capita
0,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,100M Freestyle,Gold,Hungary,9844686.0,12363.54346
1,1896,Athens,Aquatics,Swimming,"HAJOS, Alfred",HUN,Men,1200M Freestyle,Gold,Hungary,9844686.0,12363.54346
2,1896,Athens,Athletics,Athletics,"SZOKOLYI, Alajos",HUN,Men,100M,Bronze,Hungary,9844686.0,12363.54346
3,1896,Athens,Athletics,Athletics,"DANI, Nandor",HUN,Men,800M,Silver,Hungary,9844686.0,12363.54346
4,1896,Athens,Athletics,Athletics,"KELLNER, Gyula",HUN,Men,Marathon,Bronze,Hungary,9844686.0,12363.54346


# 5. Insights on the data using Python Pandas
- Create re-usable functions

In [19]:
def get_medals_gb_year_country():
    """ Group by edition and country and sum medals count.
    """
    medals_groupedBy_yearCountry = olympics.groupby(['Edition','Code']).apply(lambda country: country['Code'].count())
    return medals_groupedBy_yearCountry

def get_medals_gb_year_country_medal():
    """ Group by edition, country, medal type and sum medals count.
    """
    medals_groupedBy_yearCountryMedal = olympics.groupby(['Edition', 'Code', 'Medal']).apply(lambda country: country['Medal'].count())
    return medals_groupedBy_yearCountryMedal

def get_medals_last_10_years(countrycode):
    """ Get Gold, Silver and Bronze medals for a country for last 10 editions.
    """
    last10pics = olympics['Edition'].unique()
    yrs = pd.Series(last10pics).nlargest(10)
    df = pd.DataFrame([], columns=['Year', 'Gold', 'Silver', 'Bronze'])
    medalsdf = get_medals_gb_year_country_medal()
   
    for yr in yrs:
        medaltally = medalsdf[yr][countrycode]
        gold = 0
        silver = 0
        bronze = 0
        if 'Gold' in medaltally:
            gold = medaltally['Gold']
        if 'Silver' in medaltally:
            silver = medaltally['Silver']
        if 'Bronze' in medaltally:
            bronze =  medaltally['Bronze']
        df1 = pd.DataFrame([[yr,gold, silver, bronze]], columns=['Year', 'Gold', 'Silver', 'Bronze'])
        df = df.append(df1, ignore_index=True) 
    df = df.sort_values(by=['Year'], ascending=True)    
    df = df.reset_index()
    del df['index']
    return df

def get_correlation_medalstally():
    """ Get correlation between the medals tally and population, GDP per capita.
    """
    df = get_medals_gb_year_country()
    values  = get_all_olympic_years().values
    size = values.size
    correlations = []
    for i in range(size):
        year = values[i][0]
        df1 = df[year].to_frame(name="Tally")
        df1 = df1.reset_index()
        df2 = pd.merge(df1,dictionary, on='Code')
        corrpop = df2.corr().values[0][1]
        corrgdp = df2.corr().values[0][2]
        resp = {"Year": year, "Population":corrpop, "GDP":corrgdp}
        correlations.append(resp)
    return correlations  

def get_medals_category(countrycode, year):
    """ Get the medals count in different sports category for a country in an edition.
    """
    df = olympics[olympics['Edition'] ==  year]
    df1 = df[df['Code'] == countrycode]
    df2 = df1.groupby(['Sport']).apply(lambda country: country['Medal'].count())
    return df2

def get_medals_category_all(countrycode):  
    """ Get the medals count in different sports category for a country for last ten editions.
    """
    df1 = olympics[olympics['Code'] == countrycode]
    df2 = df1.groupby(['Sport']).apply(lambda country: country['Medal'].count())
    return df2

def get_top_ten_gold_tally(year):
    """ Get the top ten gold medal winning countries in an edition.
    """
    df = olympics[olympics['Edition'] ==  year]
    df1 = df[df['Medal'] == 'Gold']
    df2 = df1.groupby(['Code']).apply(lambda country: country['Medal'].count())
    return df2

def get_top_ten_total_tally(year):
    """ Get the top ten total medal winning countries in an edition.
    """
    df = olympics[olympics['Edition'] ==  year]
    df1 = df.groupby(['Code']).apply(lambda country: country['Medal'].count())
    return df1

def get_year_venue():
    """ Get edition venue matrix.
    """
    df = olympics[['Edition', 'City']]
    df = df.drop_duplicates()
    df = df.reset_index()
    df = df.set_index('Edition')
    del df['index']
    return df.sort_index()

def get_all_olympic_years():
    """ Get list of all olympic editions.
    """
    df = olympics['Edition']
    df = df.drop_duplicates()
    df = df.reset_index()
    del df['index']
    return df.sort_index()

def get_all_countries():
    """ Get list of all countries.
    """
    df = olympics[['Code','Country']]
    df = df.drop_duplicates()
    df = df.reset_index()
    del df['index']
    return df.sort(['Country'],ascending=[True])

def get_country_edition_data(countrycode,edition):
    """ Get data for a country and edition.
    """
    df = olympics[olympics["Code"] == countrycode]
    df1 = df[df["Edition"] == edition]
    return df1

# 6. Expose integration point with a websocket client 

In [20]:
def on_message(ws, message):
    print(message)
    msg = json.loads(message)
    cmd = msg['cmd']
    
    if cmd == 'MBY':
        country = msg['country']
        tally = get_medals_last_10_years(country)    
        tallyarray=[]
        for i, row in tally.iterrows():
            medaltally = {"Year":int(row["Year"]),
                          "Gold":int(row["Gold"]),
                          "Silver":int(row["Silver"]),
                          "Bronze":int(row["Bronze"])}
            tallyarray.append(medaltally)
        wsresponse = {}
        wsresponse["forcmd"] = "MBY" 
        wsresponse["response"] = tallyarray
        ws.send(json.dumps(wsresponse))
    elif cmd == 'MBSC':
        country = msg['country']
        year = 2008
        response = get_medals_category(country, year)
        
        ct = response.count()
        if ct > 5:
            response = response.nlargest(5)    
        
        medals = []
        categories = []
        for i, row in response.iteritems():
            categories.append(i)
            medals.append(row)   
  
        wsresponse = {}
        wsresponse["forcmd"] = "MBSC"
        wsresponse["response"] = { "categories":categories, "medals":medals}         
        ws.send(json.dumps(wsresponse))
    elif cmd == 'MBSA':
        country = msg['country']
        response = get_medals_category_all(country)
        
        ct = response.count()
        if ct > 5:
            response = response.nlargest(5)    
        
        medals = []
        categories = []
        for i, row in response.iteritems():
            categories.append(i)
            medals.append(row)   
  
        wsresponse = {}
        wsresponse["forcmd"] = "MBSA"
        wsresponse["response"] = { "categories":categories, "medals":medals}         
        ws.send(json.dumps(wsresponse))    
    elif cmd == 'T10G':
        edition = msg["edition"]
        response = get_top_ten_gold_tally(edition)
        ct = response.count()
        if ct > 10:
            response = response.nlargest(10)
        medals = []
        for i, row in response.iteritems():
            data = {"country":i,"tally":row}
            medals.append(data)  
        wsresponse = {}
        wsresponse["forcmd"] = "T10G"
        wsresponse["response"] = medals   
        print(wsresponse)
        ws.send(json.dumps(wsresponse))     
    elif cmd == 'T10M':
        year = msg["edition"]
        response = get_top_ten_total_tally(year)
        ct = response.count()
        if ct > 10:
            response = response.nlargest(10)
        medals = []
        for i, row in response.iteritems():
            data = {"country":i,"tally":row}
            medals.append(data)  
        wsresponse = {}
        wsresponse["forcmd"] = "T10M"
        wsresponse["response"] = medals   
        print(wsresponse)
        ws.send(json.dumps(wsresponse)) 
    elif cmd == 'CORR':
        corr = get_correlation_medalstally() 
        wsresponse = {}
        wsresponse["forcmd"] = "CORR"
        wsresponse["response"] = corr
        ws.send(json.dumps(wsresponse)) 
    elif cmd == 'YV':   
        yearvenue = get_year_venue()
        yearvenuearray = []
        for i in range(yearvenue.size):
            value = {"Year":yearvenue.index[i],"Venue":yearvenue.values[i].tolist()[0]}
            yearvenuearray.append(value)
        responsejson = {}
        responsejson["forcmd"]="YV"
        responsejson["response"]=yearvenuearray
        ws.send(json.dumps(responsejson))               
    elif cmd == 'DATA':
        country = msg['country']
        edition = msg['edition']
        olympicsslice = get_country_edition_data(country,edition)
        data = []
        numofcolumns = olympicsslice.columns.size
        cols = []
        values = []
        for column in olympicsslice.columns:
            cols.append(column)
        for value in olympicsslice.values:
            values.append(value.tolist()) 
        data = {"cols":cols,"vals":values}    
        responsejson = {}
        responsejson['forcmd']='DATA'
        responsejson['response']= data
        ws.send(json.dumps(responsejson)) 
    elif cmd == 'EDITIONS':
        years = get_all_olympic_years()
        yearsarray = []
        for i,row in years.iteritems():
            for value in row:
                yearsarray.append(value)
        length = len(yearsarray)
        wsresponse = []
        for i in range(length):
            year = {"text":yearsarray[i],"value":yearsarray[i]}
            wsresponse.append(year)
        responsejson = {}
        responsejson['forcmd']='EDITIONS'
        responsejson['response']= wsresponse 
        ws.send(json.dumps(responsejson)) 
    elif cmd == 'COUNTRIES':
        countries = get_all_countries()
        countriesarray = []
        codearray = []
        for i,row in countries.iteritems():
            if i=='Code':
                for value in row:
                    codearray.append(value)
            elif i=='Country':  
                for value in row:
                    countriesarray.append(value)
        length = len(codearray)
        wsresponse = []
        for i in range(length):
            country = {"text":countriesarray[i],"value":codearray[i]}
            wsresponse.append(country)
        responsejson = {}
        responsejson['forcmd']='COUNTRIES'
        responsejson['response']= wsresponse 
        ws.send(json.dumps(responsejson))  

def on_error(ws, error):
    print(error)

def on_close(ws):
    ws.send("DSX Listen End")

def on_open(ws):
    def run(*args):
        for i in range(10000):
            hbeat = '{"cmd":"Olympics DSX HeartBeat"}'
            ws.send(hbeat)
            time.sleep(100)
            
    thread.start_new_thread(run, ())


def start_websocket_listener():
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp("ws://localhost:1880/ws/orchestrate",
                              on_message = on_message,
                              on_error = on_error,
                              on_close = on_close)
    ws.on_open = on_open
    ws.run_forever()

# 7. Start websocket client

In [ ]:
start_websocket_listener()

--- request header ---
GET /ws/orchestrate HTTP/1.1
Upgrade: websocket
Connection: Upgrade
Host: localhost:1880
Origin: http://localhost:1880
Sec-WebSocket-Key: sJl74hJgsfe/v0MoVTKWYw==
Sec-WebSocket-Version: 13


-----------------------
--- response header ---
HTTP/1.1 101 Switching Protocols
Upgrade: websocket
Connection: Upgrade
Sec-WebSocket-Accept: m3+k6tnrexf3ZlQHhLe3LfIEVQM=
-----------------------
error from callback <function on_open at 0x10b5248c8>: name 'thread' is not defined
  File "/anaconda3/lib/python3.6/site-packages/websocket/_app.py", line 315, in _callback
    callback(self, *args)
  File "<ipython-input-20-9e8b8dbf2576>", line 169, in on_open
    thread.start_new_thread(run, ())


{"cmd":"Client connected"}


error from callback <function on_message at 0x10c34a378>: 'DataFrame' object has no attribute 'sort'
  File "/anaconda3/lib/python3.6/site-packages/websocket/_app.py", line 315, in _callback
    callback(self, *args)
  File "<ipython-input-20-9e8b8dbf2576>", line 136, in on_message
    countries = get_all_countries()
  File "<ipython-input-19-876731957d14>", line 113, in get_all_countries
    return df.sort(['Country'],ascending=[True])
  File "/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py", line 3614, in __getattr__
    return object.__getattribute__(self, name)
send: b'\x81\xfe\x03i\x8d]\x8a \xf6\x7f\xecO\xff>\xe7D\xafg\xaa\x02\xc8\x19\xc3t\xc4\x12\xc4s\xafq\xaa\x02\xff8\xf9P\xe23\xf9E\xafg\xaa{\xf6\x7f\xfeE\xf5)\xa8\x1a\xadl\xb2\x19\xbbq\xaa\x02\xfb<\xe6U\xe8\x7f\xb0\x00\xbce\xb3\x16\xf0q\xaa[\xaf)\xefX\xf9\x7f\xb0\x00\xbcd\xba\x10\xa1}\xa8V\xec1\xffE\xafg\xaa\x11\xb4m\xba]\xa1}\xf1\x02\xf98\xf2T\xafg\xaa\x11\xb4m\xbe\x0c\xad\x7f\xfcA\xe1(\xef\x02\xb7}\xbb\x19\xbdi\x

{"cmd":"COUNTRIES"}
{"cmd":"EDITIONS"}
{"cmd":"YV"}
{"cmd":"CORR"}


  File "/anaconda3/lib/python3.6/site-packages/websocket/_app.py", line 315, in _callback
    callback(self, *args)
  File "<ipython-input-20-9e8b8dbf2576>", line 102, in on_message
    ws.send(json.dumps(responsejson))
  File "/anaconda3/lib/python3.6/json/__init__.py", line 231, in dumps
    return _default_encoder.encode(obj)
  File "/anaconda3/lib/python3.6/json/encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "/anaconda3/lib/python3.6/json/encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "/anaconda3/lib/python3.6/json/encoder.py", line 180, in default
    o.__class__.__name__)
/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/multi.py:860: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return self._engine.get_value(s, k)
error from callback <function on_message at 0x10c34a378>: Object of type 'int64' is not JSON serializable
  Fil

send: b'\x81\xfe\x01k\xe4\xdc!\x97\x9f\xfeG\xf8\x96\xbfL\xf3\xc6\xe6\x01\xb5\xb0\xed\x11\xd0\xc6\xf0\x01\xb5\x96\xb9R\xe7\x8b\xb2R\xf2\xc6\xe6\x01\xcc\x9f\xfeB\xf8\x91\xb2U\xe5\x9d\xfe\x1b\xb7\xc6\x89r\xd6\xc6\xf0\x01\xb5\x90\xbdM\xfb\x9d\xfe\x1b\xb7\xd5\xee\x14\xea\xc8\xfcZ\xb5\x87\xb3T\xf9\x90\xaeX\xb5\xde\xfc\x03\xd4\xac\x92\x03\xbb\xc4\xfeU\xf6\x88\xb0X\xb5\xde\xfc\x16\xa3\x99\xf0\x01\xec\xc6\xbfN\xe2\x8a\xa8S\xee\xc6\xe6\x01\xb5\xb6\x89r\xb5\xc8\xfc\x03\xe3\x85\xb0M\xee\xc6\xe6\x01\xa3\xd7\xa1\r\xb7\x9f\xfeB\xf8\x91\xb2U\xe5\x9d\xfe\x1b\xb7\xc6\x9bd\xc5\xc6\xf0\x01\xb5\x90\xbdM\xfb\x9d\xfe\x1b\xb7\xd0\xee\\\xbb\xc4\xa7\x03\xf4\x8b\xa9O\xe3\x96\xa5\x03\xad\xc4\xfej\xd8\xb6\xfe\r\xb7\xc6\xa8@\xfb\x88\xa5\x03\xad\xc4\xe8\x10\xea\xc8\xfcZ\xb5\x87\xb3T\xf9\x90\xaeX\xb5\xde\xfc\x03\xd9\xa1\x98\x03\xbb\xc4\xfeU\xf6\x88\xb0X\xb5\xde\xfc\x15\xa7\x99\xf0\x01\xec\xc6\xbfN\xe2\x8a\xa8S\xee\xc6\xe6\x01\xb5\xa5\x89r\xb5\xc8\xfc\x03\xe3\x85\xb0M\xee\xc6\xe6\x01\xa4\xd5\xa1\r\xb7\x9f\xfeB\xf8\x91

{"cmd":"DATA","country":"USA","edition":2008}
{"cmd":"T10G","edition":2008}
{'forcmd': 'T10G', 'response': [{'country': 'USA', 'tally': 125}, {'country': 'CHN', 'tally': 74}, {'country': 'RUS', 'tally': 43}, {'country': 'GER', 'tally': 42}, {'country': 'KOR', 'tally': 41}, {'country': 'NED', 'tally': 40}, {'country': 'AUS', 'tally': 31}, {'country': 'GBR', 'tally': 31}, {'country': 'FRA', 'tally': 25}, {'country': 'JPN', 'tally': 23}]}
{"cmd":"T10M","edition":2008}
{'forcmd': 'T10M', 'response': [{'country': 'USA', 'tally': 315}, {'country': 'CHN', 'tally': 184}, {'country': 'AUS', 'tally': 149}, {'country': 'RUS', 'tally': 143}, {'country': 'GER', 'tally': 101}, {'country': 'KOR', 'tally': 78}, {'country': 'GBR', 'tally': 77}, {'country': 'FRA', 'tally': 76}, {'country': 'BRA', 'tally': 75}, {'country': 'ESP', 'tally': 71}]}
{"cmd":"MBSC","country":"USA"}
{"cmd":"MBSA","country":"USA"}
{"cmd":"MBY","country":"USA"}


send: b'\x81\xfe\x02T\xab%\xf3\x8c\xd0\x07\x95\xe3\xd9F\x9e\xe8\x89\x1f\xd3\xae\xe6g\xaa\xae\x87\x05\xd1\xfe\xceV\x83\xe3\xc5V\x96\xae\x91\x05\xa8\xf7\x89|\x96\xed\xd9\x07\xc9\xac\x9a\x1c\xc4\xba\x87\x05\xd1\xcb\xc4I\x97\xae\x91\x05\xc5\xbf\x87\x05\xd1\xdf\xc2I\x85\xe9\xd9\x07\xc9\xac\x9e\x13\xdf\xac\x89g\x81\xe3\xc5_\x96\xae\x91\x05\xc0\xba\xd6\t\xd3\xf7\x89|\x96\xed\xd9\x07\xc9\xac\x9a\x1c\xcb\xbc\x87\x05\xd1\xcb\xc4I\x97\xae\x91\x05\xc3\xa0\x8b\x07\xa0\xe5\xc7S\x96\xfe\x89\x1f\xd3\xbc\x87\x05\xd1\xce\xd9J\x9d\xf6\xce\x07\xc9\xac\x9bX\xdf\xac\xd0\x07\xaa\xe9\xcaW\xd1\xb6\x8b\x14\xca\xb4\x9f\t\xd3\xae\xecJ\x9f\xe8\x89\x1f\xd3\xbd\x9d\x1d\xdf\xac\x89v\x9a\xe0\xdd@\x81\xae\x91\x05\xc2\xbd\x9e\t\xd3\xae\xe9W\x9c\xe2\xd1@\xd1\xb6\x8b\x10\xc3\xf1\x87\x05\x88\xae\xf2@\x92\xfe\x89\x1f\xd3\xbd\x92\x1d\xcb\xa0\x8b\x07\xb4\xe3\xc7A\xd1\xb6\x8b\x12\xc4\xa0\x8b\x07\xa0\xe5\xc7S\x96\xfe\x89\x1f\xd3\xba\x9f\t\xd3\xae\xe9W\x9c\xe2\xd1@\xd1\xb6\x8b\x10\xc1\xf1\x87\x05\x88\xae\xf2@\x92\xfe\x89\x1f\xd3

{"forcmd": "EDITIONS", "response": [{"text": 1896, "value": 1896}, {"text": 1900, "value": 1900}, {"text": 1904, "value": 1904}, {"text": 1908, "value": 1908}, {"text": 1912, "value": 1912}, {"text": 1924, "value": 1924}, {"text": 1928, "value": 1928}, {"text": 1932, "value": 1932}, {"text": 1936, "value": 1936}, {"text": 1948, "value": 1948}, {"text": 1952, "value": 1952}, {"text": 1956, "value": 1956}, {"text": 1960, "value": 1960}, {"text": 1964, "value": 1964}, {"text": 1968, "value": 1968}, {"text": 1972, "value": 1972}, {"text": 1976, "value": 1976}, {"text": 1980, "value": 1980}, {"text": 1988, "value": 1988}, {"text": 1992, "value": 1992}, {"text": 1996, "value": 1996}, {"text": 2000, "value": 2000}, {"text": 2004, "value": 2004}, {"text": 2008, "value": 2008}, {"text": 2012, "value": 2012}, {"text": 1984, "value": 1984}, {"text": 1920, "value": 1920}]}
{"forcmd": "DATA", "response": {"cols": ["Edition", "City", "Sport", "Discipline", "Athlete", "Code", "Gender", "Event", "Meda